# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 3 2022 11:44AM,249969,10,8728400-SD,Eywa Pharma Inc.,Released
1,Nov 3 2022 11:44AM,249969,10,8728402-SD,Eywa Pharma Inc.,Released
2,Nov 3 2022 11:37AM,249968,10,SONSB0002008,"Nextsource Biotechnology, LLC",Released
3,Nov 3 2022 11:37AM,249968,10,SOTAH0000826,"Nextsource Biotechnology, LLC",Released
4,Nov 3 2022 11:19AM,249967,19,8728391,"Graystone, LLC",Released
5,Nov 3 2022 11:13AM,249966,16,SHL-8724075,"SHL Pharma, LLC",Released
6,Nov 3 2022 10:51AM,249963,19,ADV80007339,"AdvaGen Pharma, Ltd",Released
7,Nov 3 2022 10:17AM,249958,16,Emersa-8728383,"Emersa Waterbox, LLC",Released
8,Nov 3 2022 10:05AM,249952,15,VT80180164,"Virtus Pharmaceuticals, LLC",Executing
9,Nov 3 2022 10:05AM,249952,15,VT80180165,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,249963,Released,1
29,249966,Released,1
30,249967,Released,1
31,249968,Released,2
32,249969,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249963,NaN,NaN,1.0
249966,NaN,NaN,1.0
249967,NaN,NaN,1.0
249968,NaN,NaN,2.0
249969,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249884,0.0,0.0,1.0
249885,0.0,0.0,1.0
249887,0.0,0.0,1.0
249888,0.0,0.0,1.0
249889,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249884,0,0,1
249885,0,0,1
249887,0,0,1
249888,0,0,1
249889,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249884,0,0,1
1,249885,0,0,1
2,249887,0,0,1
3,249888,0,0,1
4,249889,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249884,,,1
1,249885,,,1
2,249887,,,1
3,249888,,,1
4,249889,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 3 2022 11:44AM,249969,10,Eywa Pharma Inc.
2,Nov 3 2022 11:37AM,249968,10,"Nextsource Biotechnology, LLC"
4,Nov 3 2022 11:19AM,249967,19,"Graystone, LLC"
5,Nov 3 2022 11:13AM,249966,16,"SHL Pharma, LLC"
6,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd"
7,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC"
8,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC"
11,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC"
12,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc."
57,Nov 3 2022 9:35AM,249955,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 3 2022 11:44AM,249969,10,Eywa Pharma Inc.,,,2
1,Nov 3 2022 11:37AM,249968,10,"Nextsource Biotechnology, LLC",,,2
2,Nov 3 2022 11:19AM,249967,19,"Graystone, LLC",,,1
3,Nov 3 2022 11:13AM,249966,16,"SHL Pharma, LLC",,,1
4,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd",,,1
5,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",,,1
6,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC",,1,2
7,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC",,,1
8,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc.",6,21,18
9,Nov 3 2022 9:35AM,249955,19,"GCH Granules USA, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 11:44AM,249969,10,Eywa Pharma Inc.,2,,
1,Nov 3 2022 11:37AM,249968,10,"Nextsource Biotechnology, LLC",2,,
2,Nov 3 2022 11:19AM,249967,19,"Graystone, LLC",1,,
3,Nov 3 2022 11:13AM,249966,16,"SHL Pharma, LLC",1,,
4,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd",1,,
5,Nov 3 2022 10:17AM,249958,16,"Emersa Waterbox, LLC",1,,
6,Nov 3 2022 10:05AM,249952,15,"Virtus Pharmaceuticals, LLC",2,1,
7,Nov 3 2022 9:40AM,249956,19,"NAPP Technologies, LLC",1,,
8,Nov 3 2022 9:40AM,249950,15,"Alliance Pharma, Inc.",18,21,6
9,Nov 3 2022 9:35AM,249955,19,"GCH Granules USA, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 11:44AM,249969,10,Eywa Pharma Inc.,2,,
1,Nov 3 2022 11:37AM,249968,10,"Nextsource Biotechnology, LLC",2,,
2,Nov 3 2022 11:19AM,249967,19,"Graystone, LLC",1,,
3,Nov 3 2022 11:13AM,249966,16,"SHL Pharma, LLC",1,,
4,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 11:44AM,249969,10,Eywa Pharma Inc.,2.0,NaN,NaN
1,Nov 3 2022 11:37AM,249968,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
2,Nov 3 2022 11:19AM,249967,19,"Graystone, LLC",1.0,NaN,NaN
3,Nov 3 2022 11:13AM,249966,16,"SHL Pharma, LLC",1.0,NaN,NaN
4,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 11:44AM,249969,10,Eywa Pharma Inc.,2.0,0.0,0.0
1,Nov 3 2022 11:37AM,249968,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
2,Nov 3 2022 11:19AM,249967,19,"Graystone, LLC",1.0,0.0,0.0
3,Nov 3 2022 11:13AM,249966,16,"SHL Pharma, LLC",1.0,0.0,0.0
4,Nov 3 2022 10:51AM,249963,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,499937,4.0,0.0,0.0
15,999786,40.0,32.0,6.0
16,999696,4.0,0.0,0.0
19,1749633,20.0,13.0,11.0
21,2498948,10.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,499937,4.0,0.0,0.0
1,15,999786,40.0,32.0,6.0
2,16,999696,4.0,0.0,0.0
3,19,1749633,20.0,13.0,11.0
4,21,2498948,10.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,4.0,0.0,0.0
1,15,40.0,32.0,6.0
2,16,4.0,0.0,0.0
3,19,20.0,13.0,11.0
4,21,10.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,4.0
1,15,Released,40.0
2,16,Released,4.0
3,19,Released,20.0
4,21,Released,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,0.0,6.0,0.0,11.0,0.0
Executing,0.0,32.0,0.0,13.0,0.0
Released,4.0,40.0,4.0,20.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,0.0,6.0,0.0,11.0,0.0
1,Executing,0.0,32.0,0.0,13.0,0.0
2,Released,4.0,40.0,4.0,20.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,0.0,6.0,0.0,11.0,0.0
1,Executing,0.0,32.0,0.0,13.0,0.0
2,Released,4.0,40.0,4.0,20.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()